## Data

Before running this notebook, be sure to follow these steps:

1. Download https://data.boston.gov/dataset/building-and-property-violations1/resource/800a2663-1d6a-46e7-9356-bedb70f5332c

2. Download https://data.boston.gov/dataset/census-2010-tracts/resource/6dc52eff-49e4-420c-a7df-8d758dc2da32

3. Place the files into the ``data`` folder

4. Install dependencies in the cell below

In [ ]:
# !pip install geopandas
# !pip install shapely

In [4]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

# Load GeoJSON
tracts_geojson = gpd.read_file('../data/Census_2010_Tracts.geojson')

# Load CSV
lat_lon_df = lat_lon_df = pd.read_csv('../data/boston_property_violations.csv')

In [5]:
# Convert latitude and longitude to Shapely Points
lat_lon_df['geometry'] = lat_lon_df.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)

# Convert DataFrame to GeoDataFrame
lat_lon_gdf = gpd.GeoDataFrame(lat_lon_df, geometry='geometry', crs=tracts_geojson.crs)


In [6]:
# Spatial join
result_gdf = gpd.sjoin(lat_lon_gdf, tracts_geojson, op='within', how='left')

c:\Users\ivanb\anaconda3\envs\py310\lib\site-packages\IPython\core\interactiveshell.py:3377: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


In [14]:
# Convert back to DataFrame
result_df = pd.DataFrame(result_gdf)
result_df.head()

,case_no,status_dttm,status,code,value,description,violation_stno,violation_sthigh,violation_street,violation_suffix,...,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,Shape_STAr,Shape_STLe,Shape__Area,Shape__Length
0,V517867,2020-09-28 14:27:26,Closed,102.8,NaN,Maintenance,54,NaN,Chestnut,ST,...,G5020,S,223142,0,+42.3577086,-071.0694112,2.401693e+06,7637.223404,408899.925781,3149.451618
1,V670182,2023-04-06 14:45:48,Open,102.8,NaN,Maintenance,105,NaN,Cushing,AVE,...,G5020,S,404166,0,+42.3114941,-071.0643707,4.350094e+06,9075.236281,739547.699219,3741.252990
2,V669622,2023-04-04 08:50:18,Open,111.1,NaN,Certificate of Occupancy,1,3,Clifton,ST,...,G5020,S,301664,0,+42.3229604,-071.0722753,3.246866e+06,8954.074073,552190.246094,3692.142057
3,V670166,2023-04-06 12:52:46,Open,105.1,NaN,Failure to Obtain Permit,105,NaN,Cushing,AVE,...,G5020,S,404166,0,+42.3114941,-071.0643707,4.350094e+06,9075.236281,739547.699219,3741.252990
4,V670094,2023-04-06 10:43:56,Open,102.8,NaN,Maintenance,1,11,Erie,ST,...,G5020,S,446992,0,+42.3023757,-071.0820198,4.811071e+06,10600.541287,817682.585938,4374.486230


In [32]:
violations = result_df[["status_dttm", "status", "description", "latitude", "longitude", "NAME10", "GEOID10"]]
violations = violations.dropna()
violations.head()

,status_dttm,status,description,latitude,longitude,NAME10,GEOID10
0,2020-09-28 14:27:26,Closed,Maintenance,42.357129,-71.068761,201.01,25025020101
1,2023-04-06 14:45:48,Open,Maintenance,42.313240,-71.062610,915,25025091500
2,2023-04-04 08:50:18,Open,Certificate of Occupancy,42.323738,-71.071213,906,25025090600
3,2023-04-06 12:52:46,Open,Failure to Obtain Permit,42.313240,-71.062610,915,25025091500
4,2023-04-06 10:43:56,Open,Maintenance,42.303295,-71.078780,901,25025090100


In [33]:
violations.to_csv('../data/violations_geoid.csv', index=False)

## Analysis

In [34]:
import pandas as pd

violations = pd.read_csv('../data/violations_geoid.csv')

### HeatMap of Property Violations

In [37]:
import folium
from folium.plugins import HeatMap

boston_coordinates = [42.3601, -71.0589]
heatmap_map = folium.Map(location=boston_coordinates, zoom_start=12)

In [42]:
# Extract latitude and longitude coordinates as a list of lists
coordinates = violations[['latitude', 'longitude']].values.tolist()

# Add the heatmap layer to the map
HeatMap(coordinates).add_to(heatmap_map)

# Save the heatmap to an HTML file
heatmap_map.save('../output/heatmap.html')

### 311 Calls vs Property Violations